In [ ]:
import pandas as pd
import os
import json
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
def read_bronze(ticker):
    base_path = f"data/bronze/{ticker}"
    dfs = {}
    for file_name in ["balance_sheet.json", "income_statement.json", "cashflow.json"]:
        file_path = os.path.join(base_path, file_name)
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                data = json.load(f)
            if data:
                df = pd.DataFrame.from_dict(data, orient='index')
                # Convert index to datetime if it looks like timestamp
                try:
                    df.index = pd.to_datetime(df.index.astype(int), unit='ms')
                except:
                    pass
                dfs[file_name.replace('.json', '')] = df
    return dfs

In [ ]:
def read_silver(ticker):
    file_path = f"data/silver/{ticker}.parquet"
    if os.path.exists(file_path):
        return pd.read_parquet(file_path)
    return None

In [ ]:
def format_number(x):
    if pd.isna(x):
        return "-"
    try:
        # Round to int
        val = int(round(x))
        # Format with apostrophe as thousands separator
        return f"{val:,}".replace(",", "'")
    except:
        return x

def read_gold(ticker):
    file_path = f"data/gold/{ticker}_forecast.parquet"
    if not os.path.exists(file_path):
        print(f"No Gold file found for {ticker}")
        return None
        
    df = pd.read_parquet(file_path)
    
    # Filter for Forecast only
    df_forecast = df[df['Type'] == 'Forecast'].copy()
    
    # Define ordered columns
    ordered_cols = [
        # Income Statement
        'Total Revenue', 'Cost Of Revenue', 'Operating Expense', 'Reconciled Depreciation',
        'Net Interest Income', 'Other Income Expense', 'Tax Provision',
        'Gross Profit', 'EBIT', 'EBITDA', 'EBT', 'Net Income', 'Tax Rate For Calcs',
        
        # Balance Sheet
        'Accounts Receivable', 'Accounts Payable', 'Inventory', 'Net PPE',
        'Current Assets', 'Total Non Current Assets', 'Current Liabilities',
        'Total Non Current Liabilities Net Minority Interest',
        'OtherCurrentAssets_agg', 'OtherCurrentLiabilities_agg',
        'Long Term Debt And Capital Lease Obligation', 'Goodwill And Other Intangible Assets',
        
        # Cash Flow
        'Operating Taxes', 'NOPAT', 'Gross Cash Flow',
        'Change In Inventory', 'Change In Accounts Receivable', 'Change In Accounts Payable',
        'Investment in Other Assets', 'Investment in Other Liabilities',
        'Investment in Working Capital', 'Capex', 'UFCF'
    ]
    
    # Select existing columns from the list
    existing_cols = [c for c in ordered_cols if c in df_forecast.columns]
    df_final = df_forecast[existing_cols]
    
    # Transpose: Years as columns, Accounts as rows
    df_transposed = df_final.transpose()
    
    # Format numbers
    def format_row(row):
        if row.name == 'Tax Rate For Calcs':
            return row.apply(lambda x: f"{x:.1%}" if not pd.isna(x) else "-")
        else:
            return row.apply(format_number)
            
    df_formatted = df_transposed.apply(format_row, axis=1)
    
    return df_formatted

In [ ]:
ticker = "GOOG"

print(f"--- {ticker} GOLD FORECAST ---")
gold_df = read_gold(ticker)
gold_df